Instalamos el paquete "missingno" que nos ayuda a crear un gráfico donde ver de forma visual los datos faltantes en nuestro dataset



In [2]:
# Instalación silenciosa de missingno
# (No muestra output)
%%capture
# Muestra error con el capture si sale mal la instalacion
!pip install missingno

Importamos las librerias necesarias para realizar nuestro procesamiento y limpieza de datos

In [3]:
#Importamos las librerias esenciales

import pandas as pd # para manejo de datos
import numpy as np # para operaciones numericas
import plotly as pl # para visualizaciones interactivas
import seaborn as sns # para graficos estadisticos
import missingno as msno # para visualizar datos faltantes

# Cargamos el dataset de kiva_loans.csv para trabajar

*   Elemento de lista
*   Elemento de lista

In [4]:
# Cargamos nuestro dataset principal

df = pd.read_csv("kiva_loans.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'kiva_loans.csv'

In [ ]:
# Aseguramos que sea DataFrame

df = pd.DataFrame(df)

In [ ]:
# Echamos un vistazo a las primeras 20 filas

df.head(20)

In [ ]:
# Echamos un vistazo a las primeras 20 filas

df.shape

In [ ]:

# Información general del dataset
# Tipos de datos, nulos, uso de memoria
df.info()

In [ ]:
# Contamos los valores no nulos por columna

df.count()

In [ ]:
# Estadisticas descriptivas de las columnas numericas (int y float)
df.describe()

In [ ]:
# Buscamos duplicados

df.duplicated().sum()

# Hay valores nulos?

In [ ]:
# Contamos nulos por columna, ordenados de mayor a menor
df.isna().sum(). sort_values(ascending=False)
#Suma la cantidad de valores nulos por cada columna

# Son más del 25%?

In [ ]:
import matplotlib.pyplot as plt

# Calcular % de nulos
missing = df.isna(). sum(). sort_values(ascending=False)

plt.figure(figsize=(10, 5))
missing.plot(kind="bar")
plt.title("Cantidad de datos faltantes por columna")
plt.ylabel ("Numero de valores faltantes")
plt.xlabel("Columnas")
plt.show()

In [ ]:
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
#Porcentaje de mayor a menor de datos nulos

plt.figure(figsize=(10, 5))
missing_pct.plot(kind="bar")
plt.title("Porcentaje de datos faltantes por columna")
plt.ylabel ("% de valores faltantes")
plt.xlabel("Columnas")
plt.show()

# La columna tags tiene muchos valores faltantes, debería borrarla porque no aporta información de valor

In [ ]:
msno.matrix(df)

In [ ]:
msno.bar(df)

In [ ]:
kiva_loands_df = df.copy()

# Borraremos las columnas que no nos aportan mucha información o información de valor, en este caso borraremos las columnas parthner_id, country_code, tags

In [ ]:
# cargar desde google drive from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/datasets/kiva_loans.csv')

In [ ]:
'''
Borramos 'country_code' porque es redundante ya que hay una columna 'country',
Borramos 'tags' porque al tener tantos valores faltantes no nos aporta información de valor,
Borramos 'partnet_id' porque no es una información que nos provea mucho valor,
Borramos 'borrower_genders' porque no es una información que nos provea mucho valor,
Borramos 'date' porque la columna 'posted_time' ya nos devuelve el mismo valor,
'''
cols_to_drop = ['country_code', 'tags', 'partner_id', 'borrower_genders', 'date', 'funded_time', 'region', 'use']
for col in cols_to_drop:
    # Check if the column exists in the DataFrame before dropping

    if col in kiva_loands_df.columns:
        kiva_loands_df = kiva_loands_df.drop(columns=col)


# Vemos que ahora tenemos 12 columnas y no 20

In [ ]:
kiva_loands_df.shape

In [ ]:
kiva_loands_df.head()

In [ ]:
kiva_loands_df.info()

In [ ]:
# Convertir a datetime las fechas
kiva_loands_df["posted_time"] = pd.to_datetime(kiva_loands_df["posted_time"], errors="coerce")
kiva_loands_df["disbursed_time"] = pd.to_datetime(kiva_loands_df["disbursed_time"], errors="coerce")
# Normalizar a la fecha (eliminar la hora, estableciendola en medianoche) manteniendo el tipo de datos
kiva_loands_df["posted_time"] = kiva_loands_df["posted_time"].dt.normalize()
kiva_loands_df["disbursed_time"] = kiva_loands_df["disbursed_time"].dt.normalize()

# Tomamos las columnas de tipo de dato "object" y les definimos el tipo de dato "string"

In [ ]:
kiva_loands_df.info()

In [ ]:
kiva_loands_df.head()

# Tomamos las columnas de tipo de dato "object" y les definimos el tipo de dato "string"

In [ ]:
for col in kiva_loands_df.columns:
  if kiva_loands_df[col].dtype == "object":
    kiva_loands_df[col] = kiva_loands_df[col].astype("string")

# Verificamos que haya cambiado el tipo de dato

In [ ]:
kiva_loands_df.info()

# Nos aseguramos que los datos de tipo "str" no tengan espacos innecesarios y que estén escritas en minúscula

In [ ]:
def clean_text(s):
  return s.astype(str).str.strip().str.lower()


  for col in kiva_loands_df.columns:
    if kiva_loands_df[col].dtype == "string":
      kiva_loands_df[col] == clean_text(kiva_loands_df[col])

Creamos la columna loan_type para saber si un préstamo fue desembolsado antes o después de la fecha de publicación:
·"pre_disbursed": El préstamos se desembolsó antes de publicarse (disbursed_time < posted_time)
·"post_disbursed": el préstamos se desembolsó después de publicarse (disbursed_time >= posted_time)

In [ ]:
kiva_loands_df.info()

In [ ]:
kiva_loands_df["loan_type"] = np.where(
    kiva_loands_df["disbursed_time"] < kiva_loands_df["posted_time"],
    "pre_disbursed",
    "post_disbursed"
)

In [ ]:
kiva_loands_df.head()

In [ ]:
post_disbursed_loands = kiva_loands_df[kiva_loands_df["loan_type"] == "post_disbursed"]

In [ ]:
post_disbursed_loands.head()

Creamos una nueva columna 'loan_amount' para clasifica si el prestamo es pequeño, mediano o grande

In [ ]:
# Obtener min y max reales
min_val = kiva_loands_df["loan_amount"].min()
max_val = kiva_loands_df["loan_amount"].max()
# Definir limites segun los rangos

bins = [
    min_val,     #Minimo Real
    500,
    2500,
    10000,
    max_val      #Maximo Real
]

labels = ["micro", "small", "medium", "large"]

kiva_loands_df["loan_amount_category"] = pd.cut(kiva_loands_df["loan_amount"], bins=bins, labels=labels, right=False)

# Ver resumen
print(kiva_loands_df["loan_amount_category"].value_counts())
print("Min", min_val, "Max:", max_val)




# Visualicemos la información que hemos limpiado

#Histograma:

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(kiva_loands_df["loan_amount"], bins=40, kde=False, color="skyblue")
plt.title("Distribucion de Loan Amount")
plt.xlabel("loan amount")
plt.ylabel("Frecuencia")
plt.xlim(25, 8000)
plt.show()


In [ ]:
fig,axes = plt.subplots(2,2, figsize=(14,10))
axes = axes.flatten()
#Load Amount
sns.histplot(df['loan_amount'],bins=30,kde=False,color='green',ax=axes[0])
axes[0].set_title('Distribución de Montos de Préstamos')
axes[0].set_xlabel('Monto del Préstamo')
axes[0].set_ylabel('Frecuencia')
#Funded Amount
sns.histplot(df['funded_amount'],bins=30,kde=False,color='orange',ax=axes[1])
axes[1].set_title('Distribución de Montos Financiados')
axes[1].set_xlabel('Monto Financiado')
axes[1].set_ylabel('Frecuencia')
#Term in Months
sns.histplot(df['term_in_months'],bins=30,kde=False,color='purple',ax=axes[2])
axes[2].set_title('Distribución de Términos en Meses')
axes[2].set_xlabel('Término en Meses')
axes[2].set_ylabel('Frecuencia')
#Lender Count
sns.histplot(df['lender_count'],bins=30,kde=False,color='red',ax=axes[3])
axes[3].set_title('Distribución de Cantidad de Prestamistas')
axes[3].set_xlabel('Cantidad de Prestamistas')
axes[3].set_ylabel('Frecuencia')
plt.tight_layout()
plt.show()




# Gráfico de barras:
Para datos discretos como países, sectores, tipos de loan, etc..

In [ ]:
country_totals = kiva_loands_df.groupby("country")["funded_amount"].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=country_totals.index, y=country_totals.values, palette="pastel")
plt.title("Top 10 países con el Funded Amount")
plt.xlabel("País")
plt.ylabel("Total Funded Amount")
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45)
plt.show()

In [ ]:
count_by_sector = kiva_loands_df["sector"].value_counts(). sort_values(ascending=False)

plt.figure(figsize=(10, 6))
count_by_sector.plot(kind="bar",)
plt.title("Numero de prestamos por sector")
plt.xlabel("Sector")
plt.ylabel("Numero de prestamos")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show

In [ ]:
agg = kiva_loands_df.groupby("loan_type")["lender_count"].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x="loan_type", y="lender_count", data=agg, palette="viridis")
plt.title("Promedio de lender count por loan type")
plt.xlabel("Loan Type")
plt.ylabel("Promedio Lender Count")
plt.xticks(rotation=45)
plt.show()

# Grafico de barras

In [ ]:
kiva_loands_df["month"] = kiva_loands_df["posted_time"].dt.to_period("M")
monthly_avg = kiva_loands_df.groupby("month")["loan_amount"].mean()
plt.figure(figsize=(12, 6))
monthly_avg.plot(marker="o", color="orange")
plt.title("Evolucion mensual de Loan Amount promedio")
plt.xlabel("Mes")
plt.ylabel("Promedio de Loan Amount")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Asegurate de que las fechas son datetime

kiva_loands_df["posted_time"] = pd.to_datetime(kiva_loands_df["posted_time"], errors="coerce")
# 1) Grafico de linea temporal
# Por ejemplo, montos financiados por fecha de publicacion (podrias agrupar por dias / meses)
kiva_loands_df_line = kiva_loands_df.groupby(pd.Grouper(key="posted_time", freq="M")).agg({"funded_amount": "sum"}).reset_index()
plt.figure(figsize=(10,5))
plt.plot(kiva_loands_df_line["posted_time"], kiva_loands_df_line["funded_amount"], marker="o")
plt.title("Evolucion financiados por mes")
plt.xlabel("Mes")
plt.ylabel("cantidad financiada")
plt.grid(True)
plt.show()

# Pie Chart

In [ ]:
top_countries = kiva_loands_df["country"].value_counts().nlargest(8)
plt.figure(figsize=(8, 6))
plt.pie(top_countries, labels=top_countries.index, autopct="%1.1f%%", startangle=140)
plt.title("proporcion de prestamos por pais (top 8)")
plt.show()

Choropleth map

In [ ]:
%%capture
!pip install geodatasets

In [ ]:
import geopandas as gpd
import geodatasets
import os
import requests
import zipfile

La extension .shp es la parte principal de un Shapefile, un formato de datos de Geoinformática (GIS) popular que almacena la geometría de elementos geográficos como puntos, líneas y polígonos.

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
world = gpd.read_file("ne_110m_admin_0_countries.shp")
world.head()

In [ ]:
world["NAME"]

In [ ]:
kiva_loands_df["country"] = kiva_loands_df["country"].astype(str).str.title().str.strip()
world["NAME"] = world["NAME"].astype(str).str.title().str.strip()

# Verificamos que el formato de world["NAME"] y kiva_loans_df['country'] sea el mismo

In [ ]:
kiva_loands_df["country"]

In [ ]:
country_stats = kiva_loands_df.groupby("country", as_index=False)["loan_amount"].sum()
country_stats = country_stats.rename(columns={"loan_amount": "total_loan"})

print(country_stats.index)
print(country_stats.head())


# Hacemos un left join, al dataframe world se le agrega la info del dataframe country_stats cuando coinciden, si hay un país que no existe entonces se marca como NaN

In [ ]:
merged = world.merge(country_stats, left_on="NAME", right_on="country", how="left")


In [ ]:

plt.figure(figsize=(20, 20))
merged.plot(
column="total_loan",
cmap="viridis",
legend=True,
missing_kwds={
    "color": "lightgrey",
    "edgecolor": "white",
    "hatch": "///",
    "label": "No Data"
    }
)

plt.title("total loan Amount por Pais", fontsize=18)
plt.axis("off")
plt.show()

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[20,0], zoom_start=2)

folium.GeoJson(
    merged,
    style_function=lambda feature: {
        "fillColor": "#gray" if feature["properties"]["total_loan"] is None else "#3186cc",
        "color": "#black",
        "weight": 0.5,
        "fillOpacity": 0.6
    },

    tooltip=folium.GeoJsonTooltip(
        fields=["NAME", "total_loan"],
        aliases=["Pais:", "Total Loan:"],
        localize=True,
        sticky=True,
    )
).add_to(m)
m.save("mapa_interactivo.html")
m


# Verificamos la existencia de outliers (valores atipicos), respecto a las columnas funded_amount: y loan_amount

## Regla de Tukey: La regla de Tukey es una forma sencilla de detectar valores atipicos(outliers) usando cuartiles. La idea es ver qué tan lejos están los datos de la parte "normal" del conjunto.
Q1= valor donde termina el 25% mas bajo de los datos
Q3 = valor donde termina el 75% de los datos
IQR = Q3 - Q1 --> Mide la zona donde vive la mitad central de los datos
###Un dato se considera outlier si está demasiado lejos de esa zona:
Límite inferior = Q1 -1.5 x IQR
Límite superior = Q3 + 1.5 x IQR
###Todo lo que quede fuera de ese rango se marca como valor atipico.
**Quien fue Tukey?
John Tukey fue un estadistico estadounidense, pionero en analisis y exploración de datos. Es conocido por inventar el boxplot y acuñar el termino bit en informatica

In [ ]:
categories = ["micro", "small", "medium", "large"]

for cat in categories:
  df_cat = kiva_loands_df[kiva_loands_df["loan_amount_category"] == cat]

  if df_cat.empty:
    print(f"\n{cat}: (Sin Datos)")
    continue

  q1 = df_cat["loan_amount"].quantile(0.25)
  q3 = df_cat["loan_amount"].quantile(0.75)
  iqr = q3 - q1

  # Limites para detectar outliers
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  # Filtrar outliers
  outliers = df_cat[(df_cat["loan_amount"] < lower_bound) | (df_cat["loan_amount"] > upper_bound)]
  n_outliers = len(outliers)

  print(f"\nCategoria: {cat}")
  print(f" Q1: {q1}")
  print(f" Q3: {q3}")
  print(f" IQR: {iqr}")
  print(f" Outliers: {n_outliers}")

# Gráfico de densidad

In [ ]:
for cat in labels:
  subset = kiva_loands_df[kiva_loands_df["loan_amount_category"] == cat]
  # If all values are the same, kdeplot is not informative. Use histplot for a single bar.
  plt.figure(figsize=(6,4))
  sns.kdeplot(subset["loan_amount"], fill=True, color="orange")
  plt.title(f"Densidad de prestamos - {cat}")
  plt.xlabel("Monto de prestamo")
  plt.ylabel("Densidad")
  plt.show()

In [ ]:
##folder = "/content/drive/MyDrive/datasets"
##os.makedirs(folder, exist_ok=True)

##kiva_loans_df.to_csv(f"{folder}/df_clean.csv", index=False)

In [ ]:
kiva_loands_df.to_csv("df_clean.csv")

In [ ]:
min_val = kiva_loands_df["loan_amount"].min()
max_val = kiva_loands_df["loan_amount"].max()

#Definir limites fijos (mismos que usaste para categorizar)

bounds = {
    "micro": [min_val, 500],
    "small": [500, 2500],
    "medium": [2500, 10000],
    "large": [10000, max],
}